# Lib install section

In [65]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import pandas as pd
from transformers import CLIPProcessor, CLIPModel


# Define functions and prepare dataset



## start defining some functions...

In [85]:
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


# CHANGE INPUT PICTURE PATH HERE

image = Image.open("/content/drive/MyDrive/AI/clip_interrogator/input_images/dreambooth_loulou_v_2 (11).jpg")
# image = Image.open("/content/drive/MyDrive/AI/clip_interrogator/input_images/black loulou.png")



prompt_list = ["selfie of a sks man", "selfie of a man", "portrait of a man"]

def order_prompts(model, processor, image, prompt_list):

  inputs = processor(text=prompt_list, images=image, return_tensors="pt", padding=True)
  outputs = model(**inputs)
  logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
  probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

  df1 = pd.DataFrame(probs.detach().numpy()[0])
  df2 = pd.DataFrame(prompt_list)

  df = pd.concat([df1, df2], axis=1)
  df.columns=['prob', 'prompt']

  return df.sort_values('prob', ascending=False)

In [86]:
order_prompts(model, processor, image, prompt_list)

,prob,prompt
1,0.759685,selfie of a man
2,0.167653,portrait of a man
0,0.072662,selfie of a sks man


## load dataset of famous actors

In [87]:
famous_actors = pd.read_csv('/content/drive/MyDrive/AI/clip_interrogator/datasets/famous_actors.csv')

list_of_famous_actors_names = famous_actors['title'].apply(lambda title: title.replace('_', ' ')).values
list_of_famous_actors_names

array(['Arnold Schwarzenegger', 'Adam Carolla', 'Albert Brooks', ...,
       'Jimmy Noel', 'Jeff Pope (actor)', 'Guy Teague'], dtype=object)

In [ ]:
prompt_list = []
for actor in list_of_famous_actors_names:
  prompt_list.append('selfie of actor '+actor)
prompt_list

## ...keep defining functions

In [89]:
small_prompt_list = prompt_list[:10]

In [90]:
def get_first_of_batch(model, processor, image, small_prompt_list):
  ordered_df = order_prompts(model, processor, image, small_prompt_list)
  return ordered_df.iloc[0].prompt

get_first_of_batch(model, processor, image, small_prompt_list)

'selfie of actor Adam Carolla'

In [91]:
def select_best_prompts(model, processor, image, batch_size, prompt_list):

  prompt_batches = np.array_split(prompt_list, len(prompt_list) / batch_size)

  output_prompt_list = []
  for prompt_batch in tqdm(prompt_batches):
    # print(prompt_batch)
    best_prompt = get_first_of_batch(model, processor, image, prompt_batch.tolist())
    # print(best_prompt)
    output_prompt_list.append(best_prompt)
    # break
  
  print("Selected "+str(len(output_prompt_list))+" prompts")
  return output_prompt_list

# [RUN!] Select best prompts

We rank actors in batches of `batch_size` to prevent the curse of dimensionality from ruining the probabilities associated to each prompt when `batch_size` > 50.

## Reduce from large list to small list

In [92]:
first_gen_prompts = select_best_prompts(model, processor, image, 10, prompt_list)

  0%|          | 0/905 [00:00<?, ?it/s]

Selected 905 prompts


In [93]:
second_gen_prompts = select_best_prompts(model, processor, image, 10, first_gen_prompts)

  0%|          | 0/90 [00:00<?, ?it/s]

Selected 90 prompts


In [94]:
third_gen_prompts = select_best_prompts(model, processor, image, 10, second_gen_prompts)

  0%|          | 0/9 [00:00<?, ?it/s]

Selected 9 prompts


In [95]:
third_gen_prompts

['selfie of actor Miloš Forman',
 'selfie of actor Horatio Sanz',
 'selfie of actor Dustin Farnum',
 'selfie of actor Angus T. Jones',
 'selfie of actor Jared Nathan',
 "selfie of actor Brendan O'Brien (voice actor)",
 'selfie of actor Ben Bodé',
 'selfie of actor Tom Patricola',
 'selfie of actor Zevi Wolmark']

## Final best matching actor

In [96]:
fourth_gen_prompts = select_best_prompts(model, processor, image, len(third_gen_prompts), third_gen_prompts)

  0%|          | 0/1 [00:00<?, ?it/s]

Selected 1 prompts


In [97]:
fourth_gen_prompts

['selfie of actor Horatio Sanz']

# [OPTION] Illustration section

Download a list of pictures corresponding to the list of best matching celebrities.  

Each famous actor gets the 4 first google images (to compensate for the 404 errors that sometimes happen).

In [ ]:
!pip install icrawler

In [104]:
from icrawler.builtin import GoogleImageCrawler

for google_query_to_make in tqdm(third_gen_prompts):
  google_Crawler = GoogleImageCrawler(storage = {'root_dir': r'/content/drive/MyDrive/AI/clip_interrogator/illustration_pics'})
  google_Crawler.crawl(keyword = google_query_to_make, max_num = 4, overwrite=False, file_idx_offset='auto')


  0%|          | 0/9 [00:00<?, ?it/s]

ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/The_Theatre_1904-06_Dustin_Farnum.jpg
ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Dustin_Farnum%2C_stage_actor_%28SAYRE_979%29.jpg
ERROR:downloader:Response status code 403, file https://preview.redd.it/qb7io86hiaj91.jpg
